In [1]:
import psi4
import numpy as np
import fortecubeview


r = 0.9 Angstroms
lambda_z = 0.05 atomic units
omega = 0.219 atomic units
basis = 6-31G?

In [2]:
# options dict
options_dict = {'basis': '6-31G',
               'save_jk': True, 
               'scf_type': 'pk',
               'e_convergence' : 1e-10,
               'd_convergence' : 1e-10}



molstr = """
O 0.0 0.0 0.0
H 0.0 0.0 0.9
-1 1
symmetry c1
"""

psi4.set_options(options_dict)
mol = psi4.geometry(molstr)



In [3]:
psi4.set_options({
    'CUBEPROP_TASKS': ['ORBITALS'], 
    'CUBEPROP_ORBITALS': [3,4,5,6,7,8],
})
energy, wfn = psi4.energy("scf/6-31G", molecule=mol, return_wfn=True)




Scratch directory: /tmp/
   => Libint2 <=

    Primary   basis highest AM E, G, H:  6, 6, 3
    Auxiliary basis highest AM E, G, H:  7, 7, 4
    Onebody   basis highest AM E, G, H:  -, -, -
    Solid Harmonics ordering:            Gaussian

*** tstart() called on CHEM9QDFT72ALT
*** at Tue May 27 20:06:13 2025

   => Loading Basis Set <=

    Name: 6-31G
    Role: ORBITAL
    Keyword: BASIS
    atoms 1 entry O          line   117 file /Users/jfoley19/miniconda3/envs/psi4_new/share/psi4/basis/6-31g.gbs 
    atoms 2 entry H          line    26 file /Users/jfoley19/miniconda3/envs/psi4_new/share/psi4/basis/6-31g.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference
                        1 Threads,    500 MiB Core
         --------------------------------------------------------

In [4]:
cube = psi4.cubeprop(wfn)
fortecubeview.plot()

  ==> One Electron Grid Properties (v2.0) <==

  ==> CubicScalarGrid <==

    Filepath     = .
    Total Points =            84050
    XYZ Blocking =               10
    X Points     =               41
    Y Points     =               41
    Z Points     =               50
    X Spacing    =        2.000E-01
    Y Spacing    =        2.000E-01
    Z Spacing    =        2.000E-01
    X Minimum    =       -4.000E+00
CubeViewer: listing cube files from the directory .
    Y Minimum    =       -4.000E+00
    Z Minimum    =       -4.150E+00
    X Maximum    =        4.000E+00
    Y Maximum    =        4.000E+00
    Z Maximum    =        5.650E+00

  Basis Set: 6-31G
    Blend: 6-31G
    Number of shells: 7
    Number of basis functions: 11
    Number of Cartesian functions: 11
    Spherical Harmonics?: false
    Max angular momentum: 1

Reading 6 cube files


HTML(value='\n        <style>\n           .jupyter-widgets-output-area .output_scroll {\n                heigh…

interactive(children=(Select(description='Cube files:', options=('MO    3a (3-A)', 'MO    4a (4-A)', 'MO    5a…

In [12]:
C_origin = np.loadtxt("natural_orbital_6_6_1.out")
C_disp   = np.loadtxt("natural_orbital_6_6_2.out")

print(np.linalg.norm(C_origin- C_disp))

6.042317015224304


In [6]:
# get the psi4 wavefunction as a dictionary
rhf_wfn_dict = psi4.core.Wavefunction.to_file(wfn)

# swap out orbitals in the dictionary with our orbitals from CQED-RHF
rhf_wfn_dict['matrix']['Ca'] = C_origin
rhf_wfn_dict['matrix']['Cb'] = C_origin

# Now create a new wavefunction object that has the CQED-RHF orbitals 'cqedrhf_wfn'
cqedrhf_wfn = psi4.core.Wavefunction.from_file(rhf_wfn_dict) 

# Confirm you have copied the CQED-RHF orbitals properly 
# by again getting a dic of the wavefunction and comparing
# the orbitals to the original numpy array 'C' that resulted
# from the CQED-RHF iterations
cqedrhf_wfn_dict = psi4.core.Wavefunction.to_file(cqedrhf_wfn)

assert np.allclose(cqedrhf_wfn_dict['matrix']['Ca'], C_origin, 1e-6)

   => Loading Basis Set <=

    Name: 6-31G
    Role: ORBITAL
    Keyword: BASIS
    atoms 1 entry O          line   117 file /Users/jfoley19/miniconda3/envs/psi4_new/share/psi4/basis/6-31g.gbs 
    atoms 2 entry H          line    26 file /Users/jfoley19/miniconda3/envs/psi4_new/share/psi4/basis/6-31g.gbs 



In [7]:
cube = psi4.cubeprop(cqedrhf_wfn)
fortecubeview.plot()

  ==> One Electron Grid Properties (v2.0) <==

  ==> CubicScalarGrid <==

    Filepath     = .
    Total Points =            84050
    XYZ Blocking =               10
    X Points     =               41
    Y Points     =               41
    Z Points     =               50
    X Spacing    =        2.000E-01
    Y Spacing    =        2.000E-01
    Z Spacing    =        2.000E-01
    X Minimum    =       -4.000E+00
CubeViewer: listing cube files from the directory .
    Y Minimum    =       -4.000E+00
    Z Minimum    =       -4.150E+00
    X Maximum    =        4.000E+00
    Y Maximum    =        4.000E+00
    Z Maximum    =        5.650E+00

  Basis Set: 6-31G
    Blend: 6-31G
    Number of shells: 7
    Number of basis functions: 11
    Number of Cartesian functions: 11
    Spherical Harmonics?: false
    Max angular momentum: 1

Reading 6 cube files


HTML(value='\n        <style>\n           .jupyter-widgets-output-area .output_scroll {\n                heigh…

interactive(children=(Select(description='Cube files:', options=('MO    3a (3-A)', 'MO    4a (4-A)', 'MO    5a…

In [8]:
# get the psi4 wavefunction as a dictionary
rhf_wfn_dict = psi4.core.Wavefunction.to_file(wfn)

# swap out orbitals in the dictionary with our orbitals from CQED-RHF
rhf_wfn_dict['matrix']['Ca'] = C_disp
rhf_wfn_dict['matrix']['Cb'] = C_disp

# Now create a new wavefunction object that has the CQED-RHF orbitals 'cqedrhf_wfn'
cqedrhf_wfn = psi4.core.Wavefunction.from_file(rhf_wfn_dict) 

# Confirm you have copied the CQED-RHF orbitals properly 
# by again getting a dic of the wavefunction and comparing
# the orbitals to the original numpy array 'C' that resulted
# from the CQED-RHF iterations
cqedrhf_wfn_dict = psi4.core.Wavefunction.to_file(cqedrhf_wfn)

assert np.allclose(cqedrhf_wfn_dict['matrix']['Ca'], C_disp, 1e-6)

   => Loading Basis Set <=

    Name: 6-31G
    Role: ORBITAL
    Keyword: BASIS
    atoms 1 entry O          line   117 file /Users/jfoley19/miniconda3/envs/psi4_new/share/psi4/basis/6-31g.gbs 
    atoms 2 entry H          line    26 file /Users/jfoley19/miniconda3/envs/psi4_new/share/psi4/basis/6-31g.gbs 



In [9]:
cube = psi4.cubeprop(cqedrhf_wfn)
fortecubeview.plot()

  ==> One Electron Grid Properties (v2.0) <==

  ==> CubicScalarGrid <==

    Filepath     = .
    Total Points =            84050
    XYZ Blocking =               10
    X Points     =               41
    Y Points     =               41
    Z Points     =               50
    X Spacing    =        2.000E-01
    Y Spacing    =        2.000E-01
    Z Spacing    =        2.000E-01
    X Minimum    =       -4.000E+00
    Y Minimum    =       -4.000E+00
    Z Minimum    =       -4.150E+00
    X Maximum    =        4.000E+00
    Y Maximum    =        4.000E+00
    Z Maximum    =        5.650E+00

  Basis Set: 6-31G
    Blend: 6-31G
    Number of shells: 7
    Number of basis functions: 11
    Number of Cartesian functions: 11
    Spherical Harmonics?: false
    Max angular momentum: 1

CubeViewer: listing cube files from the directory .
Reading 6 cube files


HTML(value='\n        <style>\n           .jupyter-widgets-output-area .output_scroll {\n                heigh…

interactive(children=(Select(description='Cube files:', options=('MO    3a (3-A)', 'MO    4a (4-A)', 'MO    5a…